# Mini-projet COCOMA - allocation de resssources

Le mini-projet consiste simplement à produire un notebook qui permettra d'illustrer deux protocoles pour la coordination à base d'enchères:

- les protocoles SSI (version classique et regret)

- le protocole CBAA 

On s'nspirera de la fiche exemple donnée en pdf. Pour les deux protocoles, les agents/robots doivent être situés sur un graphe, dont les arcs sont valués (distance).

In [1]:
from problem import Problem
#from utils import *
import numpy as np

## 1. Les protocoles SSI

### 1.1. Version classique

In [2]:
agents=["r1","r2"]
resources=["o1","o2","o3","o4"]

grid=np.empty([8, 8], dtype = object)

grid[2][2]="o2"
grid[2][4]="o4"
grid[4][4]="r2"
grid[5][5]="o1"
grid[5][2]="r1"
grid[7][4]="o3"
grid


array([[None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None],
       [None, None, 'o2', None, 'o4', None, None, None],
       [None, None, None, None, None, None, None, None],
       [None, None, None, None, 'r2', None, None, None],
       [None, None, 'r1', None, None, 'o1', None, None],
       [None, None, None, None, None, None, None, None],
       [None, None, None, None, 'o3', None, None, None]], dtype=object)

In [27]:
sii(grid,agents,resources)

Round:  0
	 Resource:  o1  ==> Agent:  r2 

Round:  1
	 Resource:  o2  ==> Agent:  r1 

Round:  2
	 Resource:  o3  ==> Agent:  r2 

Round:  3
	 Resource:  o4  ==> Agent:  r1 

----------- Solution ------------
Agent:  r1  :  ['o2', 'o4']
Agent:  r1  :  ['o1', 'o3']


In [26]:
def distance(x1,x2,y1,y2):
    return abs(x1-x2) + abs(y1-y2)


def initbags(n):
    m=[]
    for i in range (n):
        m.append([])
    return m


def bid(grid,r,i,agent,positions_agents):
      pos_r_x=np.where(grid == r)[0][0]
      pos_r_y=np.where(grid == r)[1][0]
      #init the nearest target with the robot's position
      nearest_pos_target_x=positions_agents[i][0][0]
      nearest_pos_target_y=positions_agents[i][0][1]
      distance_nearest_target=distance(nearest_pos_target_x,pos_r_x,nearest_pos_target_y,pos_r_y)

      for j in range(1,len(positions_agents[i])):
          pos_a_x=positions_agents[i][j][0]
          pos_a_y=positions_agents[i][j][1]
          d=distance(pos_a_x,pos_r_x,pos_a_y,pos_r_y)
          if(d<distance_nearest_target):
              nearest_pos_target_x=pos_a_x
              nearest_pos_target_y=pos_a_y
              distance_nearest_target=distance(nearest_pos_target_x,pos_r_x,nearest_pos_target_y,pos_r_y)
      return [distance_nearest_target,i]
         
def sii(grid,agents,resources):
    bags=initbags(len(agents))
    positions_agents=initbags(len(agents))
    
    cp=0
    #init position of agents
    for a in agents:
        pos_a_x=np.where(grid == a)[0][0] 
        pos_a_y=np.where(grid == a)[1][0]
        positions_agents[cp].append([pos_a_x,pos_a_y])
        cp+=1
        
    cp=0
    for r in resources:
         print("Round: ",cp)
         i=0
         bids=[]
         for agent in agents:
             bids.append(bid(grid,r,i,agents[i],positions_agents))
             i+=1
         #print(bids)
         bids=np.array(bids)
         min_bid=min(bids[:,0])
         index_winner=np.where(bids[:,0] == min_bid)[0][0]
         agent_winner=agents[index_winner]
         #print("agent_winner ",agent_winner)
         pos_r_x=np.where(grid == r)[0][0]
         pos_r_y=np.where(grid == r)[1][0]
         positions_agents[index_winner].append([pos_r_x,pos_r_y])
         #print("positions_agents ",positions_agents)
         bags[index_winner].append(r)
         print("\t Resource: ",r," ==> Agent: ",agent_winner,"\n")
         cp+=1
    i=0
    print("----------- Solution ------------")
    for b in bags:
        print("Agent: ",agents[0]," : ",b)
        i+=1
        
             
             
             

### 1.2. Version regret

## 2. Le protocoles CBAA